# Importing the dataset

In [ ]:
MovieData=pd.read_csv('/kaggle/input/imdb-india-movies/IMDb Movies India.csv',encoding='latin1')
MovieData.head(10)

In [ ]:
MovieData.info()

In [ ]:
MovieData.shape

In [ ]:
MovieData.columns

# Drop columns that are not useful for analysis

In [ ]:
MovieData = MovieData.drop(columns=['Name', 'Actor 2', 'Actor 3'])
MovieData.head()

In [ ]:
MovieData.dropna(inplace=True)

In [ ]:
MovieData.drop_duplicates(inplace=True)
MovieData.shape

# Handling Columns

In [ ]:
MovieData['Year'].unique()

In [ ]:
def handleYear(value):
    value = str(value).strip('()')
    return int(value)
MovieData['Year'] = MovieData['Year'].apply(handleYear)
MovieData['Year'].head()

In [ ]:
MovieData['Duration'].unique()

In [ ]:
def handleDuration(value):
    value=str(value).split(' ')
    value=value[0]
    return int(value)
MovieData['Duration']=MovieData['Duration'].apply(handleDuration)
MovieData['Duration'].head()

In [ ]:
MovieData['Genre'].unique()

In [ ]:
def split_genre_column(MovieData,Genre):

    MovieData['Genre1'] = MovieData[Genre].str.split(',', expand=True)[0]
    MovieData['Genre2'] = MovieData[Genre].str.split(',', expand=True)[1]
    MovieData['Genre3'] = MovieData[Genre].str.split(',', expand=True)[2]
    return MovieData

split_genre_column(MovieData,'Genre')

In [ ]:
MovieData.isna().sum()

In [ ]:
MovieData = MovieData.fillna(0)
MovieData.isna().sum()

In [ ]:
G=['Genre1','Genre2','Genre3']
for x in G:
    MovieData[x],_ = pd.factorize(MovieData[x])

MovieData = MovieData.drop(columns=['Genre'])
MovieData.head(3)

In [ ]:
MovieData['Votes'].unique()

In [ ]:
def handleVotes(value):
    value = str(value).replace(',','')
    return int(value)
MovieData['Votes'] = MovieData['Votes'].apply(handleVotes)
MovieData['Votes'].head()

# Feature Engineering

## Release Year Features

In [ ]:
MovieData['MovieAge'] = 2024 - MovieData['Year']
MovieData['MovieAge']

## Director and Actor Popularity

In [ ]:
DirectorCounts =MovieData['Director'].value_counts()
MovieData['DirectorPopularity']= MovieData['Director'].map(DirectorCounts)
ActorCounts= MovieData['Actor 1'].value_counts()
MovieData['ActorPopularity']=MovieData['Actor 1'].map(ActorCounts)

## Log Transformation of Votes

In [ ]:
MovieData['LogVotes']=np.log1p(MovieData['Votes'])
MovieData['LogVotes']

## Historical Average Ratings

In [ ]:
DirectorAvgRating = MovieData.groupby('Director')['Rating'].mean()
MovieData['DirectorAvgRating'] = MovieData['Director'].map(DirectorAvgRating)

ActorAvgRating = MovieData[['Actor 1']].stack().reset_index(name='Actor')
ActorAvgRating = ActorAvgRating.merge(MovieData[['Rating']], left_on='level_0', right_index=True)
ActorAvgRating = ActorAvgRating.groupby('Actor')['Rating'].mean()
MovieData['ActorAvgRating'] = MovieData['Actor 1'].map(ActorAvgRating)

# Visualization

## Director Influence on Ratings

In [ ]:
plt.figure(figsize=(12, 8))
TopDirectors=MovieData['Director'].value_counts().index[0:10]
sns.boxplot(data=MovieData[MovieData['Director'].isin(TopDirectors)], x='Director', y='Rating', palette='rocket')

plt.title('Box Plot of Ratings by Top Directors')
plt.xlabel('Director')
plt.ylabel('Rating')
plt.xticks(rotation=45)
plt.show()

-- Ratings are positively affected by top directors like Hrishikesh Mukherjee and Shakti Samanta, who consistently receive higher median ratings.

-- directors like Ram Gopal Varma and Vikram Bhatt exhibit a wide range of ratings, indicating variable reception and less predictable quality

-- Notable outliers for directors like Mahesh Bhatt highlight significant deviations in their movie ratings

## Actor Influence on Ratings

In [ ]:
plt.figure(figsize=(12, 8))
TopActors = MovieData['Actor 1'].value_counts().index[:10]
sns.boxplot(data=MovieData[MovieData['Actor 1'].isin(TopActors)], x='Actor 1', y='Rating', palette='mako')

plt.title('Box Plot of Ratings by Top Actors')
plt.xlabel('Actor 1')
plt.ylabel('Rating')
plt.xticks(rotation=45)
plt.show()

--Ratings are positively affected by top actors like Amitabh Bachchan and Shashi Kapoor, who consistently receive higher median ratings

-- Actors like Ajay Devgn and Sanjay Dutt show a wide range of ratings, indicating variable reception.

--Significant outliers for actors like Mithun Chakraborty and Dharmendra highlight notable deviations in their movie ratings.

## Genre Vs Rating pairplot

In [ ]:
GenreColumns=['Genre1','Genre2','Genre3','Rating']
plt.figure(figsize=(15,12))
sns.pairplot(MovieData[GenreColumns],diag_kind='kde',kind='scatter')

plt.suptitle('Pairplot of Factorized Genres and Ratings', y=1.02)
plt.show()

-- no single genre combination heavily influences the ratings.

-- The distribution of the Genre1 is heavily skewed towards a few dominant genres, while Genre2, Genre3 show more diverse but less frequent distributions.

## Distribution of Duration

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(MovieData['Duration'],bins=20,kde=True,color='lightcoral')

plt.title('Distribution of Movie Duration')
plt.xlabel('Duration (minutes)')
plt.ylabel('Frequency')
plt.show()

-- shows that the majority of movies have durations between 100 and 150 minutes

## Box Plot of Ratings by Year

In [ ]:
plt.figure(figsize=(25,9))
sns.boxplot(data=MovieData,x='Year',y='Rating',palette='Spectral')
sns.dark_palette("#69d",reverse=True,as_cmap=True)
plt.title('Box Plot of Ratings by Year')
plt.xlabel('Year')
plt.ylabel('Rating')
plt.xticks(rotation=45)
plt.show()

-- There is a general upward trend in movie ratings over the years

-- Outliers are more in the earlier years.

## Votes Distribution

In [ ]:
plt.figure(figsize=(10,6))
sns.histplot(MovieData['Votes'],bins=20,kde=True,color='blue')
plt.title('Distribution of Movie Votes')
plt.xlabel('Votes')
plt.ylabel('Frequency')
plt.xscale('log')
plt.show()

-- Most movies received less number of votes

-- There is a rapid decline in frequency as the number of votes increases.

-- Very few movies received a high number of votes.

## Violin Plot of Ratings by Genre

In [ ]:
plt.figure(figsize=(17,8))
sns.violinplot(data=MovieData,x='Genre1',y='Rating',palette='Set2')
plt.title('Violin Plot of Ratings by Genre')
plt.xlabel('Genre')
plt.ylabel('Rating')
plt.xticks(rotation=45)
plt.show()

## Joint Plot of Votes vs Rating

In [ ]:
sns.jointplot(data=MovieData,x='Votes',y='Rating',kind='hex',cmap='plasma')
plt.suptitle('Joint Plot of Votes vs Rating', y=1.02)
plt.show()

In [ ]:
MovieData['Genre1 encoded'] =round(MovieData.groupby('Genre1')['Rating'].transform('mean'),1)
MovieData['Genre2 encoded']=round(MovieData.groupby('Genre2')['Rating'].transform('mean'),1)
MovieData['Genre3 encoded'] =round(MovieData.groupby('Genre3')['Rating'].transform('mean'),1)
MovieData['Votes encoded'] =round(MovieData.groupby('Votes')['Rating'].transform('mean'), 1)
MovieData['Director encoded']= round(MovieData.groupby('Director')['Rating'].transform('mean'), 1)
MovieData['Actor 1 encoded']= round(MovieData.groupby('Actor 1')['Rating'].transform('mean'), 1)
MovieData.head()

In [ ]:
MovieData.drop(columns=['Genre1','Votes','Director','Actor 1','Genre2','Genre3'],inplace=True)
MovieData['Rating'] =round(MovieData['Rating'],1)

# Building the Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
import catboost as cb

## Train-Test Split

In [ ]:
X=MovieData.drop("Rating",axis=1)
Y=MovieData["Rating"]

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=52)

model=LinearRegression()
model.fit(X_train,Y_train)

X_test_prediction= model.predict(X_test)

mse =mean_squared_error(Y_test,X_test_prediction)
print(f"Mean Squared Error (MSE): {mse:.2f}")

r2 = r2_score(Y_test,X_test_prediction)
print(f"R-squared score: {r2:.2f}")

## Decision Trees

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_DT=DecisionTreeRegressor(random_state=42)
model_DT.fit(X_train,Y_train)
X_test_prediction_DT=model_DT.predict(X_test)

mse_DT =mean_squared_error(Y_test,X_test_prediction_DT)
print(f"Mean Squared Error (MSE): {mse_DT:.2f}")

r2_DT = r2_score(Y_test,X_test_prediction_DT)
print(f"R-squared score: {r2_DT:.2f}")

## Random Forests

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_RF=RandomForestRegressor(n_estimators=100,random_state=50)
model_RF.fit(X_train,Y_train)
X_test_prediction_RF=model_RF.predict(X_test)

mse_RF =mean_squared_error(Y_test,X_test_prediction_RF)
print(f"Mean Squared Error (MSE): {mse_RF:.2f}")

r2_RF = r2_score(Y_test,X_test_prediction_RF)
print(f"R-squared score: {r2_RF:.2f}")

## Gradient Boosting

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_GB = GradientBoostingRegressor(n_estimators=100,random_state=50)
model_GB.fit(X_train,Y_train)
X_test_prediction_GB=model_GB.predict(X_test)

mse_GB =mean_squared_error(Y_test,X_test_prediction_GB)
print(f"Mean Squared Error (MSE): {mse_GB:.2f}")

r2_GB= r2_score(Y_test,X_test_prediction_GB)
print(f"R-squared score: {r2_GB:.2f}")

## Support Vector Machines (SVM)

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_SVR = SVR(kernel='linear',C=1.0,epsilon=0.1,gamma='scale')
model_SVR.fit(X_train,Y_train)
X_test_prediction_SVR=model_SVR.predict(X_test)

mse_SVR =mean_squared_error(Y_test,X_test_prediction_SVR)
print(f"Mean Squared Error (MSE): {mse_SVR:.2f}")

r2_SVR= r2_score(Y_test,X_test_prediction_SVR)
print(f"R-squared score: {r2_SVR:.2f}")

## Neural Networks

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_NN = MLPRegressor(hidden_layer_sizes=(100,),random_state=42)
model_NN.fit(X_train,Y_train)
X_test_prediction_NN=model_NN.predict(X_test)

mse_NN =mean_squared_error(Y_test,X_test_prediction_NN)
print(f"Mean Squared Error (MSE):{mse_NN:.2f}")

r2_NN= r2_score(Y_test,X_test_prediction_NN)
print(f"R-squared score:{r2_NN:.2f}")

## XGBoost

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_XG = xgb.XGBRegressor(n_estimators=100,random_state=52)
model_XG.fit(X_train,Y_train)
X_test_prediction_XG=model_XG.predict(X_test)

mse_XG =mean_squared_error(Y_test,X_test_prediction_XG)
print(f"Mean Squared Error (MSE):{mse_XG:.2f}")

r2_XG= r2_score(Y_test,X_test_prediction_XG)
print(f"R-squared score:{r2_XG:.2f}")

## CatBoost

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.1)

model_CB = cb.CatBoostRegressor(iterations=100,random_state=42,verbose=0)
model_CB.fit(X_train,Y_train)
X_test_prediction_CB=model_CB.predict(X_test)

mse_CB =mean_squared_error(Y_test,X_test_prediction_CB)
print(f"Mean Squared Error (MSE):{mse_CB:.2f}")

r2_CB= r2_score(Y_test,X_test_prediction_CB)
print(f"R-squared score:{r2_CB:.2f}")

## Results of all models

In [ ]:
results = pd.DataFrame({
    'Model': ['Linear Regression','Decision Tree','Random Forest','Gradient Boosting',
              'Support Vector Machine','Neural Network','XGBoost','CatBoost'],
    'Mean Squared Error (MSE)': [mse*100,mse_DT*100,mse_RF*100, mse_GB*100,mse_SVR*100,mse_NN*100, mse_XG*100, mse_CB*100],
    'R-squared score (R2)': [r2*100, r2_DT*100,r2_RF*100,r2_GB*100,r2_SVR*100,r2_NN*100,r2_XG*100,r2_CB*100]
})
print(results)

## Comparision of Results.

In [ ]:
models = results['Model']
mse_scores = results['Mean Squared Error (MSE)'].round(2)
r2_scores = results['R-squared score (R2)'].round(2)
num_models = len(models)
bar_width = 0.35
index = np.arange(num_models)

In [ ]:
plt.figure(figsize=(12, 8))

plt.bar(index,mse_scores,bar_width,label='Mean Squared Error (MSE)',color='pink',alpha=0.7)
plt.bar(index +bar_width,r2_scores, bar_width,label='R-squared (R2)',color='purple',alpha=0.7)

plt.xlabel('Models')
plt.ylabel('Scores')
plt.title('Comparison of MSE and R-squared (R2) Scores for Different Models')
plt.xticks(index +bar_width/2,models,rotation=45)
plt.legend()
plt.grid(axis='y')
plt.tight_layout()
plt.show()